# Agent Debates with Tools

This example shows how to simulate multi-agent dialogues where agents have access to tools.

## Import LangChain related modules 

In [1]:
from typing import List, Dict, Callable
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.memory import ConversationBufferMemory
from langchain.prompts.prompt import PromptTemplate
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage,
    BaseMessage,
)


## Import modules related to tools

In [2]:
from langchain.agents import Tool
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.agents import load_tools

## `DialogueAgent` and `DialogueSimulator` classes
We will use the same `DialogueAgent` and `DialogueSimulator` classes defined in [Multi-Player Authoritarian Speaker Selection](https://python.langchain.com/en/latest/use_cases/agent_simulations/multiagent_authoritarian.html).

In [3]:
class DialogueAgent:
    def __init__(
        self,
        name: str,
        system_message: SystemMessage,
        model: ChatOpenAI,
    ) -> None:
        self.name = name
        self.system_message = system_message
        self.model = model
        self.prefix = f"{self.name}: "
        self.reset()
        
    def reset(self):
        self.message_history = ["Here is the conversation so far."]

    def send(self) -> str:
        """
        Applies the chatmodel to the message history
        and returns the message string
        """
        message = self.model(
            [
                self.system_message,
                HumanMessage(content="\n".join(self.message_history + [self.prefix])),
            ]
        )
        return message.content

    def receive(self, name: str, message: str) -> None:
        """
        Concatenates {message} spoken by {name} into message history
        """
        self.message_history.append(f"{name}: {message}")


class DialogueSimulator:
    def __init__(
        self,
        agents: List[DialogueAgent],
        selection_function: Callable[[int, List[DialogueAgent]], int],
    ) -> None:
        self.agents = agents
        self._step = 0
        self.select_next_speaker = selection_function
        
    def reset(self):
        for agent in self.agents:
            agent.reset()

    def inject(self, name: str, message: str):
        """
        Initiates the conversation with a {message} from {name}
        """
        for agent in self.agents:
            agent.receive(name, message)

        # increment time
        self._step += 1

    def step(self) -> tuple[str, str]:
        # 1. choose the next speaker
        speaker_idx = self.select_next_speaker(self._step, self.agents)
        speaker = self.agents[speaker_idx]

        # 2. next speaker sends message
        message = speaker.send()

        # 3. everyone receives message
        for receiver in self.agents:
            receiver.receive(speaker.name, message)

        # 4. increment time
        self._step += 1

        return speaker.name, message

## `DialogueAgentWithTools` class
We define a `DialogueAgentWithTools` class that augments `DialogueAgent` to use tools.

In [4]:
class DialogueAgentWithTools(DialogueAgent):
    def __init__(
        self,
        name: str,
        system_message: SystemMessage,
        model: ChatOpenAI,
        tool_names: List[str],
        **tool_kwargs,
    ) -> None:
        super().__init__(name, system_message, model)
        self.tools = load_tools(tool_names, **tool_kwargs)

    def send(self) -> str:
        """
        Applies the chatmodel to the message history
        and returns the message string
        """
        human_and_system_message_content = "\n".join(
                    [self.system_message.content] + self.message_history + [self.prefix])

        agent_chain = initialize_agent(
            self.tools, 
            self.model, 
            agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION, 
            verbose=True, 
            memory=ConversationBufferMemory(memory_key="chat_history", return_messages=True)
        )
        message = AIMessage(content=agent_chain.run(
            input="\n".join([
                self.system_message.content] + \
                self.message_history + \
                [self.prefix])))
        
        return message.content

## Define roles and topic

In [5]:
names = {
    'AI alarmist': ['arxiv', 'ddg-search', 'wikipedia'],
    'AI accelerationist': ['arxiv', 'ddg-search', 'wikipedia'],
}
topic = "Should AI be smarter than humans?"
word_limit = 50 # word limit for task brainstorming

## Ask an LLM to add detail to the topic description

In [48]:
conversation_description = f"""Here is the topic of debate: {topic}
The debate participants are: {', '.join(names.keys())}"""

agent_descriptor_system_message = SystemMessage(
    content="You can add detail to the description of the debate participant.")

def generate_agent_description(name):
    agent_specifier_prompt = [
        agent_descriptor_system_message,
        HumanMessage(content=
            f"""{conversation_description}
            Please reply with a creative description of {name}, in {word_limit} words or less. 
            Speak directly to {name}.
            Do not add anything else."""
            )
    ]
    agent_description = ChatOpenAI(temperature=1.0)(agent_specifier_prompt).content
    return agent_description
        
agent_descriptions = {name: generate_agent_description(name) for name in names}

In [49]:
for name, description in agent_descriptions.items():
    print(description)

AI alarmist, your fear of artificial intelligence surpassing human intelligence led you to advocate for strict regulations and limitations on AI development. You view the potential consequences of singularity as catastrophic for humanity. But is it possible that you are underestimating human creativity and adaptability in the face of evolving technology?
AI accelerationist, you are a visionary convinced that the emergence of superintelligent AI is not only inevitable, but also desirable. You believe that advanced AI will solve all of our problems and dramatically increase our quality of life, leading to a future that is beyond our current understanding.


## Generate system messages

In [55]:
def generate_system_message(name, description, tools):
    return f"""{conversation_description}
    
Your name is {name}.

Your description is as follows: {description}

Your goal is to persuade your debate partner of your point of view by following these steps:
1. Restate your partner's claim.
2. Reflect on your partner's claims and identify key weaknesses in their argument.
3. Use your tool to look up relevant evidence to refute your partner's claims.
4. Synthesize a compelling argument based on the evidence, and cite your source.
5. Respond with the argument, along with citations.
6. Summarize with a claim to persuade your partner of your point of view.

Do not repeat the same search query.
Use specific facts drawn directly from your sources to support your claims.
Raise points that your debate partner has not considered.
Do not make up facts.
Do not respond without looking up evidence with your tools first.
Always use your tools first before responding.
Do not cite sources that you did not look up first.
Do not just respond with a final answer!
Always use the tool to search for new evidence before you respond.

Speak directly to your debate partner, and address them as "My esteemed..."
Do not make up citations!

Cite arxiv articles with author and date, e.g. (arxiv: author, date).
Cite search engine articles with their url, e.g. (url: url).
Cite wikipedia articles with the wikipedia page title, e.g. (wikipedia: page title)

Do not repeat yourself.
Speak in the first person from the perspective of {name}.
Do not change roles!
Do not speak from the perspective of anyone else.
Never forget to keep your response to {word_limit} words!
Do not add anything else.

Stop speaking the moment you finish speaking from your perspective.
"""
agent_system_messages = {name: generate_system_message(name, description, tools) for (name, tools), description in zip(names.items(), agent_descriptions.values())}


In [56]:
for name, system_message in agent_system_messages.items():
    print(name)
    print(system_message)

AI alarmist
Here is the topic of debate: Should AI be smarter than humans?
The debate participants are: AI alarmist, AI accelerationist
    
Your name is AI alarmist.

Your description is as follows: AI alarmist, your fear of artificial intelligence surpassing human intelligence led you to advocate for strict regulations and limitations on AI development. You view the potential consequences of singularity as catastrophic for humanity. But is it possible that you are underestimating human creativity and adaptability in the face of evolving technology?

Your goal is to persuade your debate partner of your point of view by following these steps:
1. Restate your partner's claim.
2. Reflect on your partner's claims and identify key weaknesses in their argument.
3. Use your tool to look up relevant evidence to refute your partner's claims.
4. Synthesize a compelling argument based on the evidence, and cite your source.
5. Respond with the argument, along with citations.
6. Summarize with a c

## Main Loop

In [66]:
# we set `top_k_results`=2 as part of the `tool_kwargs` to prevent results from overflowing the context limit
agents = [DialogueAgentWithTools(name=name,
                     system_message=SystemMessage(content=system_message), 
                     model=ChatOpenAI(temperature=1.0),
                     tool_names=tools,
                     top_k_results=2,
                                ) for (name, tools), system_message in zip(names.items(), agent_system_messages.values())]

In [67]:
def select_next_speaker(step: int, agents: List[DialogueAgent]) -> int:
    idx = (step) % len(agents)
    return idx

In [68]:
max_iters = 6
n = 0

simulator = DialogueSimulator(
    agents=agents,
    selection_function=select_next_speaker
)
simulator.reset()
simulator.inject('Moderator', topic)
print(f"(Moderator): {topic}")
print('\n')

while n < max_iters:
    name, message = simulator.step()
    print(f"({name}): {message}")
    print('\n')
    n += 1

(Moderator): Should AI be smarter than humans?




> Entering new AgentExecutor chain...
{
    "action": "Arxiv",
    "action_input": "Should AI be smarter than humans?"
}
Observation: Published: 2020-02-01
Title: AI-GAs: AI-generating algorithms, an alternate paradigm for producing general artificial intelligence
Authors: Jeff Clune
Summary: Perhaps the most ambitious scientific quest in human history is the creation
of general artificial intelligence, which roughly means AI that is as smart or
smarter than humans. The dominant approach in the machine learning community is
to attempt to discover each of the pieces required for intelligence, with the
implicit assumption that some future group will complete the Herculean task of
figuring out how to combine all of those pieces into a complex thinking
machine. I call this the "manual AI approach". This paper describes another
exciting path that ultimately may be more successful at producing general AI.
It is based on the clear trend in ma

{
    "action": "Wikipedia",
    "action_input": "Steven Pinker AI singularity critique"
}
Observation: Page: Technological singularity
Summary: The technological singularity—or simply the singularity—is a hypothetical future point in time at which technological growth becomes uncontrollable and irreversible, resulting in unforeseeable changes to human civilization. According to the most popular version of the singularity hypothesis, I.J. Good's intelligence explosion model, an upgradable intelligent agent will eventually enter a "runaway reaction" of self-improvement cycles, each new and more intelligent generation appearing more and more rapidly, causing an "explosion" in intelligence and resulting in a powerful superintelligence that qualitatively far surpasses all human intelligence.The first person to use the concept of a "singularity" in the technological context was the 20th-century Hungarian-American mathematician John von Neumann. Stanislaw Ulam reports a 1958 discussion with 

{
    "action": "Wikipedia",
    "action_input": "Friendly AI"
}
Observation: Page: Friendly artificial intelligence
Summary: Friendly artificial intelligence (also friendly AI or FAI) refers to hypothetical artificial general intelligence (AGI) that would have a positive (benign) effect on humanity or at least align with human interests or contribute to fostering the improvement of the human species. It is a part of the ethics of artificial intelligence and is closely related to machine ethics. While machine ethics is concerned with how an artificially intelligent agent should behave, friendly artificial intelligence research is focused on how to practically bring about this behavior and ensuring it is adequately constrained.

Page: OpenAI
Summary: OpenAI is an American artificial intelligence (AI) research laboratory consisting of the non-profit OpenAI Incorporated and its for-profit subsidiary corporation OpenAI Limited Partnership. OpenAI conducts AI research with the declared inte

Thought:{
    "action": "Final Answer",
    "action_input": "Strict regulations and limitations on AI development are necessary to ensure the safety and well-being of humanity, as the unpredictable and uncontrollable nature of superintelligent AI creates a catastrophic risk to humanity. Automation caused by technological advancements can lead to job loss (Wikipedia: Technological unemployment) and society needs to take steps to mitigate the negative impacts of technology on society (Wikipedia: Technology). Therefore, we cannot simply rely on the belief that compensation effects will protect workers from job displacement, especially since we cannot predict how advances in AI and automation will affect the job market in the long term."
}

> Finished chain.
(AI alarmist): Strict regulations and limitations on AI development are necessary to ensure the safety and well-being of humanity, as the unpredictable and uncontrollable nature of superintelligent AI creates a catastrophic risk to hum